In [2]:
# Comman libraries 
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, List

#libararies ti import data
from tqdm import tqdm
from glob import glob
from tensorflow import data as tfd
from tensorflow import image as tfi

#libararies to stucture of models
from tensorflow.keras import Sequential
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

import tensorflow as tf
from tensorflow import keras



In [3]:

import keras_tuner as kt

In [4]:
Image_size = 160
Batch_size = 32
Epochs = 50
learning_rate = 1e-3  # for optimizer

Dataset = r"C:\Users\aishwarya.sonawane\Desktop\Image Classification\Flower Classification"
train_data = r"C:\Users\aishwarya.sonawane\Desktop\Image Classification\Flower Classification\Training Data"
test_data = r"C:\Users\aishwarya.sonawane\Desktop\Image Classification\Flower Classification\Testing Data"
vali_data = r"C:\Users\aishwarya.sonawane\Desktop\Image Classification\Flower Classification\Validation Data"


In [5]:
for i in os.listdir(train_data):
    new_loc = os.path.join(train_data,i)
    new = new_loc + '\*.jpeg'
    images = glob(new)
    print(f'{i}:',len(images))

Daisy: 1000
Lavender: 1000
Lily: 1000
Rose: 1000
Sunflower: 1000


In [6]:
for i in os.listdir(test_data):
    new_loc = os.path.join(test_data,i)
    new = new_loc + '/*.jpeg'
    images = glob(new)
    print(f'{i}:',len(images))

Daisy: 260
Lavender: 107
Lily: 271
Rose: 94
Sunflower: 135


In [7]:
class_names = sorted(os.listdir(train_data))
n_classes = len(class_names)

# Show
print(f"Total number of classes: {n_classes}")
print(f"Classes: {class_names}")

Total number of classes: 5
Classes: ['Daisy', 'Lavender', 'Lily', 'Rose', 'Sunflower']


## Preprocessing of Image

In [8]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models, datasets
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models, datasets

#device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [26]:
from PIL import Image
from glob import glob
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models, datasets

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [9]:
transform = transforms.Compose([
    transforms.Resize((300,300)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(0.3,0.4,0.4,0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.425,0.415,0.405),(0.205,0.205,0.205))
])

# Augmentation on test images not needed
transform_tests = torchvision.transforms.Compose([
    transforms.Resize((300,300)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])

In [10]:
def load_image(image_path: str) -> tf.Tensor:
    
    # Check if image path exists
    assert os.path.exists(image_path), f'Invalid image path: {image_path}'
    
    # Read the image file
    image = tf.io.read_file(image_path)
    
    # Load the image
    try:
        image = tfi.decode_jpeg(image, channels=3)
    except:
        try:
            image = tfi.decode_png(image, channels=3)
        except:
            image = None
    
    # Change the image data type
    if image is not None:
        image = tfi.convert_image_dtype(image, tf.float32)
    
    # Resize the Image
    if image is not None:
        image = tfi.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
    
    # Convert image data type to tf.float32
    if image is not None:
        image = tf.cast(image, tf.float32)
    
    return image

In [11]:
def load_dataset(root_path: str, class_names: list, trim: int=None) -> Tuple[np.ndarray, np.ndarray]:
    if trim:
        # Trim the size of the data
        n_samples = len(class_names) * trim
    else:
        # Collect total number of data samples
        n_samples = sum([len(os.listdir(os.path.join(root_path, name))) for name in class_names])

    # Create arrays to store images and labels
    images = np.empty(shape=(n_samples, IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.float32)
    labels = np.empty(shape=(n_samples, 1), dtype=np.int32)

    # Loop over all the image file paths, load and store the images with respective labels
    n_image = 0
    for class_name in tqdm(class_names, desc="Loading"):
        class_path = os.path.join(root_path, class_name)
        image_paths = list(glob(os.path.join(class_path, "*")))[:trim]
        for file_path in image_paths:
            # Load the image
            image = load_image(file_path)

            # Assign label
            label = class_names.index(class_name)

            # Store the image and the respective label
            images[n_image] = image
            labels[n_image] = label

            # Increment the number of images processed
            n_image += 1

    # Shuffle the data
    indices = np.random.permutation(n_samples)
    images = images[indices]
    labels = labels[indices]
    

    return images, labels

In [13]:
IMAGE_SIZE = 160
# Load the training dataset
X_train, y_train = load_dataset(root_path = train_data, class_names = class_names, trim=1000)  # 1000 images per class

# # Load the validation dataset
X_valid, y_valid = load_dataset(root_path = vali_data, class_names = class_names)

# Load the testing dataset
X_test, y_test = load_dataset(root_path = test_data, class_names = class_names)

Loading: 100%|███████████████████████████████████████████████████████████████████████████| 5/5 [00:04<00:00,  1.25it/s]


In [10]:
LOSS = 'sparse_categorical_crossentropy'
METRICS = ['accuracy']

BATCH_SIZE = 32 # number of samples per gradient update
EPOCHS = 50 # number of epochs to train the model

# Random Seed
random_seed = 42 
tf.random.set_seed(random_seed) 
np.random.seed(random_seed)

In [23]:
Pipeline_name = ['ResNet50V2']
Pipelines = [ResNet50V2(include_top=False, weights='imagenet', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))]

def train_pipeline(pipeline):
    # Training logic here
    history = {}  # Replace with actual training history
    return history

for name, pipeline in zip(Pipeline_name, Pipelines):
    print(f"Training: {name}\n")
    
    # Make a small model (this would be your actual training logic)
    history = train_pipeline(pipeline)  # Replace with your actual training function
    
    # Store the history
    HISTORIES[name] = history

Training: ResNet50V2



In [25]:
HISTORIES = {}

# Loop over all the pipeline
for name, Pipelines in zip(Pipeline_name, Pipelines):
     
#     # Show
    print(f"Training: {name}\n")
    
    # Make a small model
    model = Sequential([
        Pipeline,
        GlobalAveragePooling2D(),
        Dropout(0.5),
        Dense(n_classes, activation='softmax')
    ])
    
#     # Compile the model
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=Adam(1e-3),
        metrics=METRICS
    )
    
#     # Train the model
    model_history = model.fit(
        X_train[:5000], y_train[:5000],
        validation_data=(X_valid, y_valid),
        epochs=10,
        batch_size=32
    )
    
#     # Store the history
    HISTORIES[name] = model_history.history
    cls()

TypeError: 'Functional' object is not iterable

In [11]:
from tensorflow.keras.applications.resnet50 import ResNet50
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [12]:
for layer in resnet.layers:
    layer.trainable = False

In [13]:
folders = glob(r"C:\Users\aishwarya.sonawane\Desktop\Image Classification\Flower Classification\Training Data")


In [14]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
x = Flatten()(resnet.output)

In [15]:
from tensorflow.keras.models import Model
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=resnet.input, outputs=prediction)

In [16]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_pad (ZeroPadding2D)     │ (None, 230, 230, 3)       │               0 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_conv (Conv2D)           │ (None, 112, 112, 64)      │           9,472 │ conv1_pad[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_bn (BatchNormalization) │ (None, 112, 112, 64)      │             256 │ conv1_conv[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_relu (Activation)       │ (None, 112, 112, 64)      │               0 │ conv1_bn[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pool1_pad (ZeroPadding2D)     │ (None, 114, 114, 64)      │               0 │ conv1_relu[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pool1_pool (MaxPooling2D)     │ (None, 56, 56, 64)        │               0 │ pool1_pad[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_conv (Conv2D)  │ (None, 56, 56, 64)        │           4,160 │ pool1_pool[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_bn             │ (None, 56, 56, 64)        │             256 │ conv2_block1_1_conv[0][0]  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_relu           │ (None, 56, 56, 64)        │               0 │ conv2_block1_1_bn[0][0]    │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_conv (Conv2D)  │ (None, 56, 56, 64)        │          36,928 │ conv2_block1_1_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_bn             │ (None, 56, 56, 64)        │             256 │ conv2_block1_2_conv[0][0]  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_relu           │ (None, 56, 56, 64)        │               0 │ conv2_block1_2_bn[0][0]    │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_0_conv (Conv2D)  │ (None, 56, 56, 256)       │          16,640 │ pool1_pool[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_3_conv (Conv2D)  │ (None, 56, 56, 256)       │          16,640 │ conv2_block1_2_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 23,688,065 (90.36 MB)

 Trainable params: 100,353 (392.00 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [17]:

model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [19]:
training_set = train_datagen.flow_from_directory( r"C:\Users\aishwarya.sonawane\Desktop\Image Classification\Flower Classification\Training Data",
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 5000 images belonging to 5 classes.


In [20]:
test_set = test_datagen.flow_from_directory(r"C:\Users\aishwarya.sonawane\Desktop\Image Classification\Flower Classification\Testing Data",
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 958 images belonging to 5 classes.


In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

AttributeError: 'Functional' object has no attribute 'fit_generator'

------------------------------------------------

In [ ]:
def load_image(image_path: str) -> tf.Tensor:
    
    # Check if image path exists
    assert os.path.exists(image_path), f'Invalid image path: {image_path}'
    
    # Read the image file
    image = tf.io.read_file(image_path)
    
    # Load the image
    try:
        image = tfi.decode_jpeg(image, channels=3)
    except:
        try:
            image = tfi.decode_png(image, channels=3)
        except:
            image = None
    
    # Change the image data type
    if image is not None:
        image = tfi.convert_image_dtype(image, tf.float32)
    
    # Resize the Image
    if image is not None:
        image = tfi.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
    
    # Convert image data type to tf.float32
    if image is not None:
        image = tf.cast(image, tf.float32)
    
    return image

In [ ]:
def load_dataset(root_path: str, class_names: list, trim: int=None) -> Tuple[np.ndarray, np.ndarray]:

    if trim:
        # Trim the size of the data
        n_samples = len(class_names) * trim
    else:
        # Collect total number of data samples
        n_samples = sum([len(os.listdir(os.path.join(root_path, name))) for name in class_names])

    # Create arrays to store images and labels
    images = np.empty(shape=(n_samples, IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.float32)
    labels = np.empty(shape=(n_samples, 1), dtype=np.int32)

    # Loop over all the image file paths, load and store the images with respective labels
    n_image = 0
    for class_name in tqdm(class_names, desc="Loading"):
        class_path = os.path.join(root_path, class_name)
        image_paths = list(glob(os.path.join(class_path, "*")))[:trim]
        for file_path in image_paths:
            # Load the image
            image = load_image(file_path)

            # Assign label
            label = class_names.index(class_name)

            # Store the image and the respective label
            images[n_image] = image
            labels[n_image] = label

            # Increment the number of images processed
            n_image += 1

    # Shuffle the data
    indices = np.random.permutation(n_samples)
    images = images[indices]
    labels = labels[indices]
    

    return images, labels

In [ ]:

# Load the training dataset
X_train, y_train = load_dataset(root_path = train_data, class_names = class_names, trim=1000)  # 1000 images per class

# # Load the validation dataset
X_valid, y_valid = load_dataset(root_path = vali_data, class_names = class_names)

# Load the testing dataset
X_test, y_test = load_dataset(root_path = test_data, class_names = class_names)

In [ ]:
backbone_names = [
    'ResNet50V2',
]

# # Load all the backbones
backbones = [
    ResNet50V2(include_top=False, weights='imagenet', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
]

# # Freeze the weights of all the backbones
for backbone in backbones:
    backbone.trainable = False